In [1]:
from numpy.random import seed
seed(42)

import tensorflow as tf
tf.random.set_seed(42)

import numpy as np
np.random.seed(42)

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [3]:
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import OneHotEncoder

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import numpy as np
import math
import pandas as pd

#from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import InputLayer, Input
from tensorflow.python.keras.layers import Reshape, MaxPooling2D
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import skopt
from skopt import gp_minimize, forest_minimize
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_convergence
from skopt.plots import plot_objective, plot_evaluations
from skopt.plots import plot_objective_2D #, plot_histogram
from skopt.utils import use_named_args

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

/home/siderealyear/anaconda3/envs/wildfire/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/home/siderealyear/anaconda3/envs/wildfire/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [5]:
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [6]:
def log_dir_name(
    learning_rate,
    hidden_layers,
    neurons_per_layer,
    dropout_rate,
    l2_lambda,
    class_0_weight,
    class_1_weight
):

    # The dir-name for the TensorBoard log-dir.
    s = "./MLP_logs/hidden_layers{0}_neurons_per_layer{1}/"

    # Insert all the hyper-parameters in the dir-name.
    log_dir = s.format(
        learning_rate,
        hidden_layers,
        neurons_per_layer,
        dropout_rate,
        l2_lambda,
        class_0_weight,
        class_1_weight
    )

    return log_dir


def plot_metrics(history):
    metrics =  ['loss', 'auc', 'precision', 'f1']
    
    for n, metric in enumerate(metrics):
        name = metric.replace("_"," ").capitalize()
        plt.subplot(3,2,n+1)
        plt.plot(history.epoch,  history.history[metric], color=colors[0], label='Train')
        plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
        plt.xlabel('Epoch')
        plt.ylabel(name)
        
    if metric == 'loss':
        plt.ylim([0, plt.ylim()[1]])
        
    elif metric == 'auc':
        plt.ylim([0.8,1])
        
    else:
        #plt.ylim([0,1])
        plt.legend()
        
def plot_cm(labels, predictions, p=0.5):
    cm = confusion_matrix(labels, predictions > p)
    normalized_cm = np.empty([2, 2])
    normalized_cm[0][0] = cm[0][0] / (cm[0][0] + cm[0][1])
    normalized_cm[0][1] = cm[0][1] / (cm[0][0] + cm[0][1])
    normalized_cm[1][0] = cm[1][0] / (cm[1][0] + cm[1][1])
    normalized_cm[1][1] = cm[1][1] / (cm[1][0] + cm[1][1])
    plt.figure(figsize=(5,5))
    sns.heatmap(normalized_cm, annot=True)
    plt.title('Confusion matrix @{:.2f}'.format(p))
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')

    print('No fire (True Negatives): ', cm[0][0])
    print('False alarms (False Positives): ', cm[0][1])
    print('Fires missed (False Negatives): ', cm[1][0])
    print('Fires detected (True Positives): ', cm[1][1])
    print('Total fires: ', np.sum(cm[1]))
    
def plot_roc(name, labels, predictions, **kwargs):
    fp, tp, _ = sklearn.metrics.roc_curve(labels, predictions)

    plt.plot(100*fp, 100*tp, label=name, linewidth=2, **kwargs)
    plt.xlabel('False positives [%]')
    plt.ylabel('True positives [%]')
    #plt.xlim([-0.5,20])
    #plt.ylim([80,100.5])
    plt.grid(True)
    ax = plt.gca()
    ax.set_aspect('equal')
    
def f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    
    return f1_val

In [7]:
data_file = '../data/stratified_training_data/1992-2015_training_data_added_features_n500000_ks_pval0.3.1.csv'
validation_file = '../data/stratified_training_data/1992-2015_training_data_added_features_n500000_ks_pval0.3.2.csv'
test_file = '../data/stratified_training_data/1992-2015_training_data_added_features_n500000_ks_pval0.3.3.csv'

# Datatypes for dataframe loading
dtypes = {
    'lat': float,
    'lon': float,
    'weather_bin_year': int,
    'weather_bin_month': int,
    'weather_bin_day': int,
    'air.2m': float,
    'apcp': float,
    'rhum.2m': float,
    'dpt.2m': float,
    'pres.sfc': float,
    'uwnd.10m': float,
    'vwnd.10m': float,
    'veg': float,
    'vis': float,
    'ignition': float,
    'mean.air.2m': float,
    'mean.apcp': float,
    'mean.rhum.2m': float,
    'mean.dpt.2m': float,
    'mean.pres.sfc': float,
    'mean.uwnd.10m': float,
    'mean.vwnd.10m': float,
    'mean.veg': float,
    'mean.vis': float,
    'max.air.2m': float,
    'max.apcp': float,
    'max.rhum.2m': float,
    'max.dpt.2m': float,
    'max.pres.sfc': float,
    'max.uwnd.10m': float,
    'max.vwnd.10m': float,
    'max.veg': float,
    'max.vis': float,
    'min.air.2m': float,
    'min.apcp': float,
    'min.rhum.2m': float,
    'min.dpt.2m': float,
    'min.pres.sfc': float,
    'min.uwnd.10m': float,
    'min.vwnd.10m': float,
    'min.veg': float,
    'min.vis': float,
    'total_fires': float

}

# Features to use during training 
features = [
    'lat',
    'lon',
    'weather_bin_month',
    'veg',
    'ignition',
    'mean.air.2m',
    'mean.apcp',
    'mean.rhum.2m',
    'mean.dpt.2m',
    'mean.pres.sfc',
    'mean.uwnd.10m',
    'mean.vwnd.10m',
    'mean.vis',
    'max.air.2m',
    'max.apcp',
    'max.rhum.2m',
    'max.dpt.2m',
    'max.pres.sfc',
    'max.uwnd.10m',
    'max.vwnd.10m',
    'max.vis',
    'min.air.2m',
    'min.apcp',
    'min.rhum.2m',
    'min.dpt.2m',
    'min.pres.sfc',
    'min.uwnd.10m',
    'min.vwnd.10m',
    'min.vis',
    'total_fires'
]

features_to_scale = [
    'lat',
    'lon',
    'veg',
    'mean.air.2m',
    'mean.apcp',
    'mean.rhum.2m',
    'mean.dpt.2m',
    'mean.pres.sfc',
    'mean.uwnd.10m',
    'mean.vwnd.10m',
    'mean.vis',
    'max.air.2m',
    'max.apcp',
    'max.rhum.2m',
    'max.dpt.2m',
    'max.pres.sfc',
    'max.uwnd.10m',
    'max.vwnd.10m',
    'max.vis',
    'min.air.2m',
    'min.apcp',
    'min.rhum.2m',
    'min.dpt.2m',
    'min.pres.sfc',
    'min.uwnd.10m',
    'min.vwnd.10m',
    'min.vis',
    'total_fires'
]

metrics = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'), 
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc'),
    f1
]

dim_learning_rate = Real(
    low=1e-5, 
    high=1e-1, 
    prior='log-uniform',
    name='learning_rate'
)

dim_hidden_layers = Integer(
    low=2,
    high=30, 
    name='hidden_layers'
)

dim_neurons_per_layer = Integer(
    low=10, 
    high=200, 
    name='neurons_per_layer'
)

dim_dropout_rate = Real(
    low=0.01, 
    high=0.5, 
    name='dropout_rate'
)

dim_l2_lambda = Real(
    low=0.0001, 
    high=1, 
    name='l2_lambda'
)

dim_class_0_weight = Real(
    low=0.1,
    high=10,
    name='class_0_weight'
)

dim_class_1_weight = Real(
    low=5,
    high=20,
    name="class_1_weight"
)

dimensions = [
    dim_learning_rate,
    dim_hidden_layers,
    dim_neurons_per_layer,
    dim_dropout_rate,
    dim_l2_lambda,
    dim_class_0_weight,
    dim_class_1_weight
]

default_parameters = [0.001, 2, 30, 0.5, 0.1, 0.5, 15]

path_best_model = '../trained_models/best_skopt_MLP.keras'
best_fraction_incorrect = 1.0

In [8]:
data = pd.read_csv(data_file, dtype=dtypes)
validation = pd.read_csv(validation_file, dtype=dtypes)
test = pd.read_csv(test_file, dtype=dtypes)

In [9]:
# Pull out columns for X (data to train with) and Y (value to predict)
training_data = data[features]
validation_data = validation[features]
test_data = test[features]

In [10]:
# One hot encode month

column_names = [
    'January',
    'February',
    'March',
    'April',
    'May',
    'June',
    'July',
    'August',
    'Septermber',
    'October',
    'November',
    'December'
]


onehot_encoder = OneHotEncoder(sparse=False)

# Training data
month = np.array(training_data['weather_bin_month']).reshape(-1, 1)
onehot_month = onehot_encoder.fit_transform(month)

training_data.drop('weather_bin_month', axis=1, inplace=True)
onehot_month_df = pd.DataFrame(onehot_month, columns=column_names)
training_data = pd.concat([training_data, onehot_month_df], axis=1)

# Validation data
month = np.array(validation_data['weather_bin_month']).reshape(-1, 1)
onehot_month = onehot_encoder.fit_transform(month)

validation_data.drop('weather_bin_month', axis=1, inplace=True)
onehot_month_df = pd.DataFrame(onehot_month, columns=column_names)
validation_data = pd.concat([validation_data, onehot_month_df], axis=1)

# Test data data
month = np.array(test_data['weather_bin_month']).reshape(-1, 1)
onehot_month = onehot_encoder.fit_transform(month)

test_data.drop('weather_bin_month', axis=1, inplace=True)
onehot_month_df = pd.DataFrame(onehot_month, columns=column_names)
test_data = pd.concat([test_data, onehot_month_df], axis=1)

/home/siderealyear/anaconda3/envs/wildfire/lib/python3.6/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/siderealyear/anaconda3/envs/wildfire/lib/python3.6/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/siderealyear/anaconda3/envs/wildfire/lib/python3.6/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

In [11]:
training_data.head()

,lat,lon,veg,ignition,mean.air.2m,mean.apcp,mean.rhum.2m,mean.dpt.2m,mean.pres.sfc,mean.uwnd.10m,...,March,April,May,June,July,August,Septermber,October,November,December
0,34.18678,-118.8088,25.5,0.0,289.170681,0.000000,57.576653,279.735122,96650.870000,0.064065,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,37.03086,-119.0176,43.3,0.0,294.386004,0.000122,33.640771,277.393281,79824.146797,-0.407725,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,37.49919,-119.8433,33.9,0.0,287.263178,0.139648,60.458148,279.509951,89032.984219,-0.223786,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,36.23206,-118.5007,46.9,0.0,267.102028,0.000318,45.363947,256.344916,76906.623375,0.438386,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,32.91990,-114.8649,3.5,0.0,289.117580,0.000506,26.899587,269.320998,99661.673406,1.417608,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 449258 entries, 0 to 449257
Data columns (total 41 columns):
lat              449258 non-null float64
lon              449258 non-null float64
veg              449258 non-null float64
ignition         449258 non-null float64
mean.air.2m      449258 non-null float64
mean.apcp        449258 non-null float64
mean.rhum.2m     449258 non-null float64
mean.dpt.2m      449258 non-null float64
mean.pres.sfc    449258 non-null float64
mean.uwnd.10m    449258 non-null float64
mean.vwnd.10m    449258 non-null float64
mean.vis         449258 non-null float64
max.air.2m       449258 non-null float64
max.apcp         449258 non-null float64
max.rhum.2m      449258 non-null float64
max.dpt.2m       449258 non-null float64
max.pres.sfc     449258 non-null float64
max.uwnd.10m     449258 non-null float64
max.vwnd.10m     449258 non-null float64
max.vis          449258 non-null float64
min.air.2m       449258 non-null float64
min.apcp         449258 non-

In [13]:
# Form np arrays of labels and features.
train_labels = np.array(training_data.pop('ignition'))
bool_train_labels = train_labels != 0
val_labels = np.array(validation_data.pop('ignition'))
test_labels = np.array(test_data.pop('ignition'))

train_features = np.array(training_data)
val_features = np.array(validation_data)
test_features = np.array(test_data)

In [14]:
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)

val_features = scaler.transform(val_features)
test_features = scaler.transform(test_features)

print('Training labels shape:', train_labels.shape)
print('Validation labels shape:', val_labels.shape)
print('Test labels shape:', test_labels.shape)

print('Training features shape:', train_features.shape)
print('Validation features shape:', val_features.shape)
print('Test features shape:', test_features.shape)

Training labels shape: (449258,)
Validation labels shape: (449257,)
Test labels shape: (449258,)
Training features shape: (449258, 40)
Validation features shape: (449257, 40)
Test features shape: (449258, 40)


In [15]:
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
# weight_for_0 = (1 / no_ignition_count)*(total)/2.0 
# weight_for_1 = (1 / ignition_count)*(total)/2.0

# class_weight = {0: weight_for_0, 1: weight_for_1}

# print('Weight for class 0: {:.2f}'.format(weight_for_0))
# print('Weight for class 1: {:.2f}'.format(weight_for_1))

ignition_count = sum(train_labels)
no_ignition_count = len(train_labels) - ignition_count

initial_bias = np.log([ignition_count/no_ignition_count])
output_bias = tf.keras.initializers.Constant(initial_bias)

EPOCHS = 20
BATCH_SIZE = 500
STEPS_PER_EPOCH = (len(train_features) * 0.1) // BATCH_SIZE
VALIDATION_STEPS = (len(val_features) * 0.1) // BATCH_SIZE

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

In [16]:
def make_model(
    output_bias,
    learning_rate,
    hidden_layers,
    neurons_per_layer,
    dropout_rate,
    l2_lambda
):    
    # Define the keras model
    model = keras.Sequential()
    
    # Add input layer
    model.add(keras.layers.Dense(
        neurons_per_layer, 
        activation = 'relu', 
        input_dim = train_features.shape[-1],
    ))

    # Add fully connected hidden layers
    for i in range(hidden_layers):
        model.add(keras.layers.Dense(
            neurons_per_layer,
            bias_initializer=keras.initializers.VarianceScaling(
                scale=1.0,
                mode='fan_in', 
                distribution='normal', 
                seed=None
            ),
            kernel_regularizer=keras.regularizers.l2(l2_lambda),
            activation = 'relu')
        )
    
    # Add dropout layer
    model.add(keras.layers.Dropout(dropout_rate))
    
    # Add output layer
    model.add(keras.layers.Dense(
        1, 
        activation = 'sigmoid', 
        bias_initializer = output_bias
    ))

    model.compile(
        optimizer=keras.optimizers.Adam(lr=learning_rate),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics
    )

    return model


@use_named_args(dimensions=dimensions)
def fitness(
    learning_rate,
    hidden_layers,
    neurons_per_layer,
    dropout_rate,
    l2_lambda,
    class_0_weight,
    class_1_weight
):
    """
    Hyper-parameters:
    learning_rate:     Learning-rate for the optimizer.
    num_dense_layers:  Number of dense layers.
    num_dense_nodes:   Number of nodes in each dense layer.
    activation:        Activation function for all layers.
    """

    class_weight = {0: class_0_weight, 1: class_1_weight}
    
    # Print the hyper-parameters.
    print('learning rate: {0:.1e}'.format(learning_rate))
    print('hidden layers:', hidden_layers)
    print('neurons per layer:', neurons_per_layer)
    print('dropout rate: {}'.format(np.round(dropout_rate,2)))
    print('l2 lambda: {0:.1e}'.format(l2_lambda))
    print('class weight: {}, {}'.format(np.round(class_weight[0],1), np.round(class_weight[1],2)))
    print()
    
    # Create the neural network with these hyper-parameters.
    model = make_model(
        output_bias,
        learning_rate = learning_rate,
        hidden_layers = hidden_layers,
        neurons_per_layer = neurons_per_layer,
        dropout_rate = dropout_rate,
        l2_lambda = l2_lambda
    )
    
    model.summary()
    print()

    # Dir-name for the TensorBoard log-files.
    log_dir = log_dir_name(
        learning_rate,
        hidden_layers,
        neurons_per_layer,
        dropout_rate,
        l2_lambda,
        class_0_weight,
        class_1_weight
    )
    
    # Create a callback-function for Keras which will be
    # run after each epoch has ended during training.
    # This saves the log-files for TensorBoard.
    # Note that there are complications when histogram_freq=1.
    # It might give strange errors and it also does not properly
    # support Keras data-generators for the validation-set.
    callback_log = TensorBoard(
        log_dir=log_dir,
        histogram_freq=0,
        write_graph=True,
        write_grads=False,
        write_images=False
    )
   
    # Use Keras to train the model.
    history = model.fit(
        train_features,
        train_labels,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        steps_per_epoch=STEPS_PER_EPOCH,
        callbacks = [early_stopping],
        validation_data=(val_features, val_labels),
        validation_steps=VALIDATION_STEPS,
        class_weight=class_weight,
        workers=8
    )

    # Get fraction incorrect on the validation-set
    # after the last training-epoch.
          
    val_fp = history.history['val_fp'][-1]
    val_fn = history.history['val_fn'][-1]
    val_tp = history.history['val_tp'][-1]
    val_tn = history.history['val_tn'][-1]
          
    fraction_incorrect = (val_fn /(val_fn + val_tp + K.epsilon())) + (val_fp / (val_fp + val_tn + K.epsilon()))
    
    print()
    print("Validation fraction incorrect: {0:.2}".format(fraction_incorrect))
    print()

    # Save the model if it improves on the best-found performance.
    # We use the global keyword so we update the variable outside
    # of this function.
    global best_fraction_incorrect

    # If the classification accuracy of the saved model is improved ...
    if fraction_incorrect < best_fraction_incorrect:
        # Save the new model to harddisk.
        model.save(path_best_model)
        
        # Update the classification accuracy.
        best_fraction_incorrect = fraction_incorrect

    # Delete the Keras model with these hyper-parameters from memory.
    del model
    
    # Clear the Keras session, otherwise it will keep adding new
    # models to the same TensorFlow graph each time we create
    # a model with a different set of hyper-parameters.
    K.clear_session()
    
    # NOTE: Scikit-optimize does minimization so it tries to
    # find a set of hyper-parameters with the LOWEST fitness-value.
    # Because we are interested in the HIGHEST classification
    # accuracy, we need to negate this number so it can be minimized.
    return fraction_incorrect

In [ ]:
search_result = gp_minimize(
    func=fitness,
    dimensions=dimensions,
    acq_func='EI', # Expected Improvement.
    n_calls=20,
    x0=default_parameters
)

learning rate: 1.0e-03
hidden layers: 2
neurons per layer: 30
dropout rate: 0.5
l2 lambda: 1.0e-01
class weight: 0.5, 15

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                1230      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                930       
_________________________________________________________________
dense_2 (Dense)              (None, 30)                930       
_________________________________________________________________
dropout (Dropout)            (None, 30)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 31        
Total params: 3,121
Trainable params: 3,121
Non-trainable params: 0
________________________________________________________________

 44500/449258 [=>............................] - ETA: 9s - loss: 0.5672 - tp: 1206.0000 - fp: 8805.0000 - tn: 34032.0000 - fn: 457.0000 - accuracy: 0.7919 - precision: 0.1205 - recall: 0.7252 - auc: 0.8440 - f1: 0.2062 - val_loss: 0.0521 - val_tp: 1349.0000 - val_fp: 11376.0000 - val_tn: 31496.0000 - val_fn: 279.0000 - val_accuracy: 0.7381 - val_precision: 0.1060 - val_recall: 0.8286 - val_auc: 0.8654 - val_f1: 0.1869
Validation fraction incorrect: 0.44

learning rate: 1.5e-02
hidden layers: 7
neurons per layer: 158
dropout rate: 0.3
l2 lambda: 4.5e-01
class weight: 1.1, 11.89

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 158)               6478      
_________________________________________________________________
dense_1 (Dense)              (None, 158)               25122     
_________________________________________________________________

 44258/449258 [=>............................] - ETA: 27s - loss: 0.8921 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 42678.0000 - fn: 1580.0000 - accuracy: 0.9643 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.4994 - f1: 0.0000e+00 - val_loss: 0.0891 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 42872.0000 - val_fn: 1628.0000 - val_accuracy: 0.9634 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5000 - val_f1: 0.0000e+00Epoch 00011: early stopping

Validation fraction incorrect: 1.0

learning rate: 2.2e-04
hidden layers: 6
neurons per layer: 134
dropout rate: 0.04
l2 lambda: 7.2e-01
class weight: 9.4, 5.01

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 134)               5494      
_________________________________________________________________
dense_1 (Dense)              (None, 134)               18090     
________

Train on 449258 samples, validate on 449257 samples
Epoch 1/20
 44258/449258 [=>............................] - ETA: 52s - loss: 1.4162 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 42678.0000 - fn: 1580.0000 - accuracy: 0.9643 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5003 - f1: 0.0000e+00 - val_loss: 0.1428 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 42872.0000 - val_fn: 1628.0000 - val_accuracy: 0.9634 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5000 - val_f1: 0.0000e+00Epoch 00011: early stopping

Validation fraction incorrect: 1.0

learning rate: 1.5e-05
hidden layers: 29
neurons per layer: 54
dropout rate: 0.05
l2 lambda: 6.2e-01
class weight: 3.9, 19.75

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 54)                2214      
_________________________________________________________________
dense_1 (D

Train on 449258 samples, validate on 449257 samples
Epoch 1/20
 44258/449258 [=>............................] - ETA: 40s - loss: 802.6221 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 42678.0000 - fn: 1580.0000 - accuracy: 0.9643 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5061 - f1: 0.0000e+00 - val_loss: 78.8002 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 42872.0000 - val_fn: 1628.0000 - val_accuracy: 0.9634 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5000 - val_f1: 0.0000e+00Epoch 00011: early stopping

Validation fraction incorrect: 1.0

learning rate: 7.4e-04
hidden layers: 26
neurons per layer: 139
dropout rate: 0.23
l2 lambda: 1.3e-02
class weight: 9.4, 13.45

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 139)               5699      
_________________________________________________________________
dense_

...

 44500/449258 [=>............................] - ETA: 15s - loss: 0.6066 - tp: 1239.0000 - fp: 10825.0000 - tn: 32012.0000 - fn: 424.0000 - accuracy: 0.7472 - precision: 0.1027 - recall: 0.7450 - auc: 0.8226 - f1: 0.1797 - val_loss: 0.0572 - val_tp: 1284.0000 - val_fp: 11510.0000 - val_tn: 31362.0000 - val_fn: 344.0000 - val_accuracy: 0.7336 - val_precision: 0.1004 - val_recall: 0.7887 - val_auc: 0.8442 - val_f1: 0.1771
Validation fraction incorrect: 0.48

learning rate: 6.4e-02
hidden layers: 16
neurons per layer: 67
dropout rate: 0.35
l2 lambda: 8.0e-01
class weight: 2.7, 15.26

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 67)                2747      
_________________________________________________________________
dense_1 (Dense)              (None, 67)                4556      
______________________________________________________________

 44258/449258 [=>............................] - ETA: 38s - loss: 1.4588 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 42678.0000 - fn: 1580.0000 - accuracy: 0.9643 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.4961 - f1: 0.0000e+00 - val_loss: 0.1468 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 42872.0000 - val_fn: 1628.0000 - val_accuracy: 0.9634 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5000 - val_f1: 0.0000e+00Epoch 00011: early stopping

Validation fraction incorrect: 1.0

learning rate: 6.7e-04
hidden layers: 30
neurons per layer: 163
dropout rate: 0.17
l2 lambda: 8.9e-02
class weight: 9.3, 11.28

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 163)               6683      
_________________________________________________________________
dense_1 (Dense)              (None, 163)               26732     
______

Train on 449258 samples, validate on 449257 samples
Epoch 1/20
 44258/449258 [=>............................] - ETA: 1:54 - loss: 1.6635 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 42678.0000 - fn: 1580.0000 - accuracy: 0.9643 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5057 - f1: 0.0000e+00 - val_loss: 0.1679 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 42872.0000 - val_fn: 1628.0000 - val_accuracy: 0.9634 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5000 - val_f1: 0.0000e+00Epoch 00011: early stopping

Validation fraction incorrect: 1.0

learning rate: 1.1e-05
hidden layers: 8
neurons per layer: 192
dropout rate: 0.21
l2 lambda: 7.9e-01
class weight: 0.6, 9.18

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 192)               7872      
_________________________________________________________________
dense_1 (D

 44500/449258 [=>............................] - ETA: 39s - loss: 807.8803 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 42866.0000 - fn: 1634.0000 - accuracy: 0.9633 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5176 - f1: 0.0000e+00 - val_loss: 79.0421 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 42872.0000 - val_fn: 1628.0000 - val_accuracy: 0.9634 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5185 - val_f1: 0.0000e+00Epoch 19/20


 44500/449258 [=>............................] - ETA: 55s - loss: 769.3868 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 42837.0000 - fn: 1663.0000 - accuracy: 0.9626 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5157 - f1: 0.0000e+00 - val_loss: 75.2654 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 42872.0000 - val_fn: 1628.0000 - val_accuracy: 0.9634 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5017 - val_f1: 0.0000e+00
Validation fraction incorrect: 1.0

learning rate: 8.6e-02
hidden layers: 18
neurons per layer: 31
dropout rate: 0.39
l2 lambda: 5.7e-01
class weight: 6.0, 18.15

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 31)                1271      
_________________________________________________________________
dense_1 (Dense)              (None, 31)                992       
________________________________

In [ ]:
dim_names = [
    'learning_rate',
    'hidden_layers',
    'neurons_per_layer',
    'dropout_rate',
    'l2_lambda',
    'class_0_weight',
    'class_1_weight'
]

In [ ]:
fig, ax = plot_objective(result=search_result, dimension_names=dim_names)

In [ ]:
space = search_result.space
winning_hyperparams = space.point_to_dict(search_result.x)
winning_hyperparams

In [ ]:
# 0.001, 2, 30, 0.5, 0.1, 0.5, 15 defaults

In [ ]:
learning_rate = winning_hyperparams['learning_rate'] / 10
#learning_rate = 0.0001
hidden_layers = winning_hyperparams['hidden_layers']
units_per_layer = winning_hyperparams['neurons_per_layer']
dropout_rate = winning_hyperparams['dropout_rate']
l2_lambda = winning_hyperparams['l2_lambda']
#l2_lambda = 0.005
class_0_weight = winning_hyperparams['class_0_weight']
#class_0_weight = 0.6
class_1_weight = winning_hyperparams['class_1_weight']

initial_bias = np.log([ignition_count/no_ignition_count])
output_bias = tf.keras.initializers.Constant(initial_bias)
    
class_weight = {0: class_0_weight, 1: class_1_weight}

EPOCHS = 30
BATCH_SIZE = 100000
STEPS_PER_EPOCH = (len(training_data) * 1) // BATCH_SIZE
VALIDATION_STEPS = (len(validation_data) * 1) // BATCH_SIZE

In [ ]:
# Define the keras model
model = keras.Sequential()

# Add input layer
model.add(keras.layers.Dense(
    units_per_layer, 
    activation = 'relu', 
    input_dim = train_features.shape[-1],
))

# Add fully connected hidden layers
for i in range(hidden_layers):
    model.add(keras.layers.Dense(
        units_per_layer,
        bias_initializer=keras.initializers.VarianceScaling(
            scale=1.0,
            mode='fan_in', 
            distribution='normal', 
            seed=None
        ),
        kernel_regularizer=keras.regularizers.l2(l2_lambda),
        activation = 'relu')
    )

# Add dropout layer
model.add(keras.layers.Dropout(dropout_rate))

# Add output layer
model.add(keras.layers.Dense(
    1, 
    activation = 'sigmoid', 
    bias_initializer = output_bias
))

model.compile(
    optimizer=keras.optimizers.Adam(lr=learning_rate),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=metrics
)

In [ ]:
history = model.fit(
    train_features,
    train_labels,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks = [early_stopping],
    validation_data=(val_features, val_labels),
    validation_steps=VALIDATION_STEPS,
    class_weight=class_weight,
    workers=8
)

model.save('../trained_models/best_MLP.keras')

In [ ]:
plot_metrics(history)

In [ ]:
train_predictions = model.predict(train_features, batch_size=len(train_features))
test_predictions = model.predict(test_features, batch_size=len(test_features))

results = model.evaluate(
    test_features,
    test_labels,
    batch_size=BATCH_SIZE, 
    verbose=0
)

for name, value in zip(model.metrics_names, results):
    print(name, ': ', value)

plot_cm(test_labels, test_predictions)

In [ ]:
plot_roc("Train", train_labels, train_predictions, color=colors[0])
plot_roc("Test", test_labels, test_predictions, color=colors[0], linestyle='--')

plt.legend(loc='lower right')

plt.show()